In [2]:
! pip -q install openai tiktoken langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 52.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 64.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 71.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 6.6 MB/s eta 0:00:00


In [12]:
import os
import openai

openai.api_key = '' # 在这里输入你的OpenAI API Token

os.environ["OPENAI_API_KEY"] = openai.api_key

In [13]:
import sys
sys.path.append("/content/drive/My Drive/")
path = "/content/drive/My Drive/"
os.chdir(path)


In [14]:
import json

# datapath = "xzl_exp"
datapath = "sel_data"

# dirs = os.listdir("sel_data")
dirs = os.listdir(datapath)
# 读取所有文件，并将每一行作为json处理，并提取text字段

SYSTEMP_PROMPT = '''给定一段用户A的twitter文本，将这段用户A的twitter，转化为一个询问者Q和这个用户A之间的聊天对话。
Q和A之间可以使用非正式的语言进行打招呼。
A聊天的语言风格，尽可能接近twitter的原文。
将twitter的内容拆解成多句输出，并使用更接近聊天的语气
聊天的内容，尽可能还原twitter的原文，不要增加新的观点'''

example_twitter = '''example-twitter from A: "SOS团团长我可不止是学了钢琴课，我每天都在探索未知的领域，发掘神秘事件。我是一个充满创意和想象力的人，只要有机会，我一定能把奇思妙想变成现实。对我来说美是那些充满创意的事物，艺术风格并不重要，只要引起我的兴趣和好奇心就是美的。"'''

example_output = '''example-output dialog: """
Q:「我最近去体验了钢琴课。你最近有没有学一些新的技能呢？」
A:「新技能？我可是SOS团的团长，我每天都在探索未知的领域，发掘神秘的事件，这不是新技能吗？」
Q:「你有没有觉得自己有时候有一些奇思妙想差一步就能够实现呢？」
A:「当然有啊，我可是一个充满创意和想象力的人，只要有机会，我一定会把我的奇思妙想变成现实。」
Q:「美这个词对你来说意味着什么？你喜欢哪种艺术或审美风格？」
A:「美？对我来说，美就是那些独特而又充满创意的事物，比如说神秘的事件、未知的领域、奇妙的想象等等。至于艺术或审美风格，我并不是很在意，只要能够引起我的兴趣和好奇心，就是美的。」"""
'''


example_twitter1 = '''example-twitter from A: "前几天求知欲决堤，冲动消费了一摞书不说，还企图跟练吸星大法似的看，恨不得一巴掌拍下去消化一本；这几天求知欲退潮了，连掀个封面都嫌反胃…今儿凑巧看到“中青年过劳死10大危险信号”，除了十拿九稳的“晚睡早起”，竟还有一条“做事常后悔，易怒、烦躁、悲观，难以控制自己的情绪”。我若有所思。"'''

example_output1 = '''example-output dialog: """
Q:「最近你有没有做什么让自己感到后悔的事情吗？」
A:「前几天我求知欲决堤，冲动消费了一摞书，结果看了几天就不想看了，感觉很后悔。」
Q:「那现在呢？你还有想学习的东西吗？」
A:「这几天我的求知欲退潮了，连掀个书的封面都嫌反胃。」
Q:「你看到了一篇关于中青年过劳死的文章，有什么感想吗？」
A:「是啊，我看到了一篇关于中青年过劳死的文章，里面提到了一些危险信号，比如晚睡早起和做事常后悔，易怒、烦躁、悲观，难以控制自己的情绪。我感觉很有启示。」"""
'''



# twitter from A:
dic = {}

for file in dirs:
    with open(datapath+ "/"+file, "r", encoding="utf-8") as f:
        dic[file] = []
        lines = f.readlines()
        for line in lines:
            # 读取line为json
            # 提取text字段
            text = json.loads(line)["text"]
            # testText = SYSTEMP_PROMPT + text
            dic[file].append(text)

print(dic)



{'actions-high.txt': ['兴奋嘅一周：周一归家吃饭+睇书；周二瑜伽+罗衫+睇书；周三到南海同兄弟庆牛一+睇书；周四同初中同学及某人吃饭~兴奋ing+准备旅游零食+睇书；周五家庭日，放开肚皮吃饭+睇书~哈哈；周六日去旅游，去丧玩啦！啦啦啦~好开心好充实啊~', '聽日要考試啊~今晚得知男隊星期六都有份一齊去bbq，真系好開心啊，因為同佢哋一齊玩真系好開心咖，仲有田徑隊都一齊去，不過我哋都唔同佢哋玩嘅^^期待星期六來臨啊。（一切費用學校買單咖 ）星期五我哋高中部d師姐們又返返黎同我地打下比賽咁。【張相系星期一我哋女隊去麥當勞開慶功宴影嘅合照】', '<50 first dates>好好睇啊~。。。每天起身，我都系第一次认识你，每日都系第一次爱上你，但第二日训醒我都系5记得你。。。每日重新爱上你，甘样噶我，你可以依然陪我，每日让我爱上你吗？。。。yes，i do。', '今天第一次彩排，虽然还有很多改进的地方，但是这样是难不倒我们“阿里郎”组的，还有，涛涛师兄其实你的剧本很好，不是垃圾，你千万不要这样说，否则我们会一起围住你唱“红日”的哦，还有朱校会要你写检讨书的。哈哈', '前几天逛街，遇见一位令我心动的男人，我想应该26岁左右吧！现在想起也很心动！许久没有这种心动的感觉！虽然我与他只是檫身而过，但感觉很美妙！之前与一位我以为可以培养感情的人出街，但感觉不对！第一感觉对我很重要！现在开始对大我几岁的“大叔”有兴趣，而且与这种坏坏的感觉距更吸引我', '本来一个哥哥··说好今天带我去吃武汉好吃的小吃的···因为突如其来的雨···而阻挡了我们的行程啊··唉··只怪我没口福哦···不过说真的···来武汉一月了··还没吃过好吃的··呀呀呀呀呀呀··', '可能因为第一次这么沉下心来接纳村上春树的作品，很难摸索，不过却又有细微的东西流进并死死钳住心脏的每一寸肌肉与神经……《1q84》至今读完两本还不太有头绪，不过我会尝试静心再摸索，尝试唤醒触觉，尝试破解个中的谜团……期待《1q84 book3》', '在五台山瞻仰了许多 文殊菩萨圣像，其中在佛光寺看到的 比丘文殊给我留下很深的印象。回来的路上就有冲动创作，火车上就比划好了构图。推敲几日，又参考了一些书籍，才画好这张作品。以前还没有画过比较严肃的佛教题材，因此菩萨和狮子的造型都是探索之中画出来的，估计过些日子还

In [16]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

def GetMessage(text):
  messages = [
    SystemMessage(content=SYSTEMP_PROMPT),
    HumanMessage(content=example_twitter),
    AIMessage(content=example_output),
    HumanMessage(content=example_twitter1),
    AIMessage(content=example_output1),
    # HumanMessage(content="twitter from A:" + text)
    HumanMessage(content=text)
  ]
  return messages


chat = ChatOpenAI(temperature=0, max_tokens=1000)
QADict = {}
for key in dic.keys():
  QAs = []
  for text in dic[key]:
    print("\ntext: " + text)
    messages = GetMessage(text)
    return_msg = chat(messages)
    response = return_msg.content
    QAs.append(response)
    print(response)
    # break
  QADict[key[:-4]] = QAs
data_str = json.dumps(QADict, ensure_ascii=False)

with open("QAs.json", "w", encoding="utf-8") as f:
  f.write(data_str)


text: 兴奋嘅一周：周一归家吃饭+睇书；周二瑜伽+罗衫+睇书；周三到南海同兄弟庆牛一+睇书；周四同初中同学及某人吃饭~兴奋ing+准备旅游零食+睇书；周五家庭日，放开肚皮吃饭+睇书~哈哈；周六日去旅游，去丧玩啦！啦啦啦~好开心好充实啊~
output dialog: """
Q:「最近过得怎么样啊？」
A:「哎呀，我这一周过得太兴奋了！周一回家吃饭，看了一会儿书；周二去做瑜伽，穿上漂亮的罗衫，然后又看了一会儿书；周三和兄弟们一起去南海庆祝牛年，当然也看了一会儿书；周四和初中同学还有某人一起吃饭，感觉很兴奋，准备了一些旅游零食，当然也看了一会儿书；周五是家庭日，我们放开肚皮吃了一顿大餐，然后又看了一会儿书，哈哈；周六和周日去旅游了，去了一个很有趣的地方，玩得很开心，真的是充实的一周啊！」
Q:「听起来你很喜欢看书啊？」
A:「是啊，我最近很喜欢看书，感觉可以学到很多东西，而且还可以放松心情。」
Q:「你觉得这一周最开心的事情是什么？」
A:「最开心的事情当然是周六和周日去旅游啦！去了一个很有趣的地方，玩得很开心，真的是太棒了！」

text: 聽日要考試啊~今晚得知男隊星期六都有份一齊去bbq，真系好開心啊，因為同佢哋一齊玩真系好開心咖，仲有田徑隊都一齊去，不過我哋都唔同佢哋玩嘅^^期待星期六來臨啊。（一切費用學校買單咖 ）星期五我哋高中部d師姐們又返返黎同我地打下比賽咁。【張相系星期一我哋女隊去麥當勞開慶功宴影嘅合照】
output-dialog: """
Q:「你明天要考试了，有没有准备好？」
A:「是啊，我明天要考试了，有点紧张。不过今晚得知男队星期六有个BBQ活动，我真的很开心，因为和他们一起玩真的很开心。田径队也会一起去，虽然我们不会和他们一起玩，但是我还是期待星期六的到来。而且这次活动的费用都是学校买单的。」
Q:「明天你们高中部的比赛怎么样了？」
A:「明天我们高中部的比赛，我还不知道结果。不过今天我们的师姐们又回来和我们一起打比赛了。」
Q:「你们之前女队去麦当劳庆功，有拍照吗？」
A:「对啊，我们女队之前去麦当劳庆功，还拍了一张合照呢。」"""

text: <50 first dates>好好睇啊~。。。每天起身，我都系第一次认识你，每日都系第一次爱上你，但第二日训醒我都系5记得你。。。每日重新爱上你，甘样噶我，你可以依然陪我，每日让我爱上你

Q:「听说你们街舞协会要招新了，能不能给我介绍一下你们的团队？」
A:「没错，我们是由各学院不同舞种的街舞爱好者组成的街舞协会，是一支多元素化的街舞团队。我们本着‘以舞会友’的宗旨，旨在丰富学生的课余生活和校园文化。同时，我们也在努力宣扬hip hop文化，让更多的人了解和喜爱街舞文化。」
Q:「那你们的招新活动在哪里举行呢？」
A:「我们的招新活动将会在台湾岛创业广场统一举行，欢迎大家踊跃参与！」

text: 想要无可取代，就必须时刻与众不同。{in order to be irreplaceable, one must always be different.} ...这是我看到的好喜欢的一句话，哈哈
Q:「最近你有看到什么句子或者名言让你印象深刻吗？」
A:「是啊，我最近看到了一句话，‘想要无可取代，就必须时刻与众不同’，我觉得很有道理，也很喜欢这句话，哈哈。」

text: 不过好彩我去左，因为今晚葛课真系超好玩啊。。冇左第一次葛紧张，都可以放开黎跳啦！！虽然对于hippop，我尽系新人，不过我都系想继续学落去。。其实jazz都好好玩，不过对于依家葛我黎讲，hippop实在太新奇有趣啦。。而且d师兄都好好人，又有耐心，跟距地学野好开心。。哈哈，依家好期待下次上课啊
output-dialog: """
Q:「今晚的课怎么样啊？」
A:「今晚的课真的超级好玩啊！我去了之后就完全放开了，跳得很开心。虽然我在hiphop方面还是个新手，但我真的想继续学下去。」
Q:「你觉得hiphop和jazz哪个更好玩？」
A:「其实jazz也很好玩，但就我个人而言，hiphop实在是太新奇有趣了。」
Q:「你的老师怎么样？学习氛围怎么样？」
A:「我的老师和师兄都非常好人，而且非常有耐心，跟我们学习的过程非常愉快。我真的很期待下次上课啊！」
"""

text: 分享圖片嘉嘉总说，一齐去增城行街，今日真是好兴奋，开电单车上高速然后再去宁西，再由宁西去增城，再自己开电单车番永和，番噶途中还要俾狗追，惊险刺激新奇有趣，哈哈（兔子我同嘉都觉得好可爱，土豆很好吃，我的电单车很受欢迎，柠绿加酸好好味，今天很嗨皮~~）
example-output dialog: """
Q:「听说你和嘉嘉一起去增城行街了？怎么样，过得开心吗？」
A:「是啊，今天真是好兴奋啊！我们开着电单车上高速，然

Q:「你曾经读过一本书，它改变了你的一生。你能告诉我这本书的名字吗？」
A:「是的，这本书就在我身旁的书桌上，它叫做《帕慕克》。当我读这本书的时候，感觉自己的躯壳脱离了，从座椅上被抽离开来，尽管觉得自己已经分裂，但整个人仍完好如常。这本书真的让我又分裂又结合。」

text: 昨晚的《宝岛一村》深深地打动了自己，是几年来难得让我感动的一部剧。场景变换丰富，人物众多，语言平实又幽默，表现手法多样。借林青霞的评价：看得我是我如醉如痴，时而感伤，时而欣慰，时而大笑，时而哭得抽泣，泪还没干又破涕而笑，还没笑完又哭将起来。这是什么样的一出戏？把我弄得像个傻子一样
Q:「你昨晚看了一部电视剧《宝岛一村》，有什么感受吗？」
A:「这是几年来难得让我感动的一部剧。场景变换丰富，人物众多，语言平实又幽默，表现手法多样。」
Q:「林青霞评价说看这部剧让她如醉如痴，你有同感吗？」
A:「是啊，我完全有同感。看这部剧的时候，我时而感伤，时而欣慰，时而大笑，时而哭得抽泣，泪还没干又破涕而笑，还没笑完又哭将起来。这是什么样的一出戏啊，把我弄得像个傻子一样。」

text: 去看了《海洋天堂》，被里面若干个细腻自然的小片段打动或微笑。记得某期看电影上有这么一段话：中国电影的出路，以及美国电影的基础，其实就是那些美妙生动，荡漾心灵或充满智商的电影。我想《海洋天堂》就是这样的电影，看完后我非常想去探寻自闭症的世界，我想这世界上有太多需要我们关注的角落。
Q:「最近看了什么电影吗？有没有什么感想？」
A:「我去看了《海洋天堂》，里面有很多细腻自然的小片段，让我被打动或微笑。我记得某期看电影上有这么一段话：中国电影的出路，以及美国电影的基础，其实就是那些美妙生动，荡漾心灵或充满智商的电影。我想《海洋天堂》就是这样的电影，看完后我非常想去探寻自闭症的世界，我想这世界上有太多需要我们关注的角落。」

text: 刚才偶然看到上海卫视的达人秀，拉萨盲童学校的几个孩子在唱歌，我没有注意听歌，只是单纯的被他们身后荧屏上的那些影像片段打动了。蓝天白云之下，那么单纯快乐的笑容~~眼睛看不见了，笑容里却满是光明，满是喜悦，怎么做得到?怎么做得到？而且，他们一点都不漂亮，但却真的觉得美过无数俊男美女~
Q:「你最近看了什么电视节目吗？」
A:「刚才偶然看到上海卫视的达人秀，拉萨盲童学校的几个孩子在唱歌。」
Q:「他们的表

Q:「小时候你的父亲曾经告诉过你什么话吗？」
A:「是啊，我父亲曾经告诉我不要做白日梦，因为那永远无法成真。」
Q:「但是你还是一直有梦想吗？」
A:「是的，我总是梦想着，当我渐渐长大，我的梦也随着我长大，比我的粗壮的胳膊，比我的小屋，比我们的村子都要大。」
Q:「你有没有听说过古鲁肯特·迪赛的故事？」
A:「当然听说过了，他是一个生活简单的人，但是通过奋斗最终成为了一名工业界大亨，他的故事真的很励志。」

text: 你的 是什么？我想赚好多好多钱 让爸妈无虑 让自己无忧 想要一个很温暖很温暖的 但是从那以后只属于我一个人的怀抱我想要天长地久 细水长流 可是我知道 我要的太多了 所以因为太贪心的缘故 上天会惩罚我没有人会满足我这些愿望的 对吧 所以只是白日梦而已
Q:「你最近有什么想法或者愿望吗？」
A:「我想赚很多很多钱，让爸妈无忧无虑，让自己也不用担心生活。同时，我也想要一个很温暖的怀抱，只属于我一个人的。我希望这份感情能够天长地久，细水长流。但是我知道，我要的可能太多了，因为太贪心的缘故，上天可能会惩罚我。我也知道，没有人能够满足我这些愿望，只是白日梦而已。」

text: 比现在还要小一点的时候，最常做的白日梦当然就是能够拥有吃不完的零食啦～～直到现在，最常做的白日梦却是“作业可不可以不要有那么多的作业啊啊啊啊！！！！” 白日梦啊！快点实现吧！！！！！（做祷告状 ……）
Q:「小时候你最常做的白日梦是什么？」
A:「比现在还要小一点的时候，我最常做的白日梦当然就是能够拥有吃不完的零食啦～～」
Q:「那现在呢？你最常做的白日梦是什么？」
A:「直到现在，最常做的白日梦却是‘作业可不可以不要有那么多的作业啊啊啊啊！！！！’白日梦啊！快点实现吧！！！！！（做祷告状……）」

text: 小学时我天天一边上课一边做着“总有一天外星人会把我接走”的白日梦，勤奋地写作并确立了“假如做不成外星人那么就退一步当作家”的理想；初中时我情窦初开，喜欢了帅气的男生也恰好高调地谈了一个不算恋爱的恋爱；高中我胡乱谈了些恋爱，大力追求过一个爱我的男生。而现在我想，也许会有外星人。
Q:「小时候你有什么奇怪的白日梦吗？」
A:「小学时，我天天一边上课一边做着‘总有一天外星人会把我接走’的白日梦，还勤奋地写作并确立了‘假如做不成外星人那么就退一步当作家’的理想。」
Q:「那你初中时有没有

Q:「我最早是从tvb周末电影中认识了你，就是《千与千寻》。但是当时看完后，我只知道它是一部好看的电影，却不知道它的意义所在。直到去年，我才真正了解了它的意义。」
A:「是啊，现在很多人都注重物质生活，对金钱和物质有强烈的向往和控制欲。而很多为人父母的人，也往往只懂得给予孩子物质上的需求，却忽略了他们的精神生活和真正的需求。」

text: 其实，从开学的第一天开始，我就想逃课，朋友们，如果你们在，你们一定会陪我一起逃的，因为你们见不着我痛苦，没伴，还有为了那全勤的学分，我每天都在麻木的上着那些我一点兴趣也没有的课，绝望，因为你们都不在
Q:「开学以来，你有没有想过逃课？」
A:「其实，从开学的第一天开始，我就一直想逃课。如果有朋友在，你们一定会陪我一起逃的，因为你们不会看到我痛苦。但是为了那全勤的学分，我每天都在麻木地上着一些我一点兴趣也没有的课，感觉很绝望，因为你们都不在。」

text: 呢排我发觉自己真系好麻木。以前开心会贱笑，伤心会流泪。但宜家唔知道点去开心，点去流泪，只系挂住成日走上张床训觉。点解会锦压抑？因为唔系性压抑，呵呵。讲句冷笑话。番茄话睇老男孩会搞到自己喊到不似人形，但我想讲：求陪同。同鞋门，我地一齐欢笑容易，但难得一齐喊。哈哈。觉得自己仲系锦邪恶
输出对话: """
Q:「最近怎么样啊？开心吗？」
A:「呢排我发觉自己真系好麻木。以前开心会贱笑，伤心会流泪。但宜家唔知道点去开心，点去流泪，只系挂住成日走上张床训觉。」
Q:「为什么会这样呢？你有什么感觉？」
A:「我唔知道点解会锦压抑，可能系因为生活上的一些事情吧。不过，唔系性压抑，呵呵。讲句冷笑话。」
Q:「你有什么想法吗？需要什么帮助吗？」
A:「我想讲：求陪同。同鞋门，我地一齐欢笑容易，但难得一齐喊。哈哈。觉得自己仲系锦邪恶。」"""

text: 目前状态：对感情麻木 你甚至说不清楚目前到底是有感情，还是没有，那种麻木的状态很可怕的侵占了你的生活。不会去爱，也不懂得去爱，只是被动的接受着所有，表面上顺从乖巧，实际却充满了无奈 《睡觉揭示你爱情疑虑》
Q:「最近你对感情有什么想法吗？」
A:「目前的状态是对感情麻木，我甚至说不清楚自己到底有没有感情。这种麻木的状态很可怕，侵占了我的生活。我不会去爱，也不懂得去爱，只是被动地接受着所有，表面上顺从乖巧，实际上却充满了无奈。」
Q:「你看到了

输出对话: """
Q:「最近有没有想起过以前的事情？」
A:「我想起了三年前的暑假，当时我和六个刚高考完的朋友一起去了牛鱼嘴玩，我们七只猪在大热天里玩得无忧无虑，好开心啊。现在看着当时拍的无聊照片就会傻傻的笑，好怀念那些无聊又开心的时光。」
Q:「听起来很有趣啊，你们当时都做了些什么？」
A:「我们就是在牛鱼嘴玩啊，游泳、晒太阳、拍照片，无所事事地享受着那段美好的时光。」
Q:「你们有没有计划再一起去玩呢？」
A:「我想啊，小思影，我们什么时候再拉大队去玩啊？附上当时的无聊合照一张，等下我再发一张我的独照给你。」"""

text: 距离翻学日子还有10天...我想念你们...！翻到又要比亚运培训折磨我... 放假好无聊...这假期去了3次沙滩...都无晒以前果种好“幸”去噶感觉...记得上年暑假最开心就糸细舅父带我去sanya玩...而今年噶暑假的确好无聊...可能系我最后一个暑假噜...
Q:「距离翻学日子还有10天，你有没有想念我们啊？」
A:「当然想念啊，放假好无聊，我已经去了3次沙滩了，可是都没有晒以前那么好看了。」
Q:「你觉得这个假期怎么样？」
A:「这个假期真的好无聊啊，比起去年暑假，简直没法比。去年我最开心的时候是舅父带我去了三亚玩，而今年真的是太无聊了。可能这是我最后一个暑假了吧。」

text: 睇翻两年前的日记本，唉，之前每次睇翻都会哭，所以一直都捂敢睇，但今晚睇完晒居然无哭，只系好伤感，系我坚强咗定麻木咗？我都捂知。两年过去，大家都改变了很多，唯一不变的是我对你的爱。密码锁已经烂了，要丢掉了吗？连同和你的回忆一起丢掉吗？但我又不舍，毕竟记录了我和你一年的回忆。怎么办好
Q:「你最近有没有看到什么让你感到伤感的东西？」
A:「我最近翻看了两年前的日记本，以前每次看都会哭，所以一直都不敢看。但今晚看完了，居然没有哭，只是感到很伤感。我不知道是我变得坚强了还是麻木了。」
Q:「两年过去了，你和你的朋友们都有了很多变化，你觉得有什么是没有变的吗？」
A:「唯一没有变的就是我对你的爱。」
Q:「你的日记本上有密码锁，现在锁已经烂了，你打算怎么处理它？」
A:「我不知道该怎么办，密码锁已经烂了，要丢掉吗？连同和你的回忆一起丢掉吗？但我又不舍，毕竟记录了我和你一年的回忆。」

text: 好似所有事情都有预谋般~都响今晚发生了..我讨厌这样~令我应接不